# Export Fusion model to ONNX

In [1]:
import numpy as np
import onnxruntime as rt
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, RobustScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier, XGBRegressor, DMatrix, train as train_xgb
from skl2onnx.common.data_types import FloatTensorType, Int64TensorType
from skl2onnx import convert_sklearn, to_onnx, update_registered_converter
from skl2onnx.common.shape_calculator import (
    calculate_linear_classifier_output_shapes,
    calculate_linear_regressor_output_shapes
)

from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost
import joblib
import pickle
from scipy.special import softmax

In [2]:
pipeline_full = joblib.load('../Saved_Models/EF1a_F.joblib')

/Users/steven/Documents/GitHub.nosync/mids-210-radiology-triage-models-spring24/.conda/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator SimpleImputer from version 1.3.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/steven/Documents/GitHub.nosync/mids-210-radiology-triage-models-spring24/.conda/lib/python3.11/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator OrdinalEncoder from version 1.3.2 when using version 1.4.1.post1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/steven/Documents/GitHub.nosync/mids-210-radi

In [3]:
pipeline_full

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('ordinal',
                                                  Pipeline(steps=[('missing',
                                                                   SimpleImputer(fill_value=-1,
                                                                                 strategy='constant')),
                                                                  ('Ordinal',
                                                                   OrdinalEncoder()),
                                                                  ('Scale',
                                                                   RobustScaler(with_centering=False))]),
                                                  ['pain', 'acuity']),
                                                 ('ratio',
                                                  Pipeline(steps=[('mean',
                                                                   SimpleImputer()),
                                                                  ('Scale',
                                                                   RobustScaler(with_centering=False))]),
                                                  ['temper...
                               feature_types=None, gamma=1.6, gpu_id=None,
                               grow_policy=None, importance_type='gain',
                               interaction_constraints=None, learning_rate=0.01,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=0.0,
                               max_depth=3, max_leaves=None,
                               min_child_weight=17, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=1750, n_jobs=-1, num_class=5, ...))])

In [5]:
# Define data type columns
ordinal_cols = ['pain', 'acuity']
ratio_cols   = ['temperature', 'heartrate', 'resprate', 'o2sat', 'sbp', 'dbp',
               'no_findings_prob_img','atelectasis_prob_img','cardiomegaly_prob_img','lung_opacity_prob_img','plueral_effusion_prob_img',
               'no_findings_prob_txt','atelectasis_prob_txt','cardiomegaly_prob_txt','lung_opacity_prob_txt','plueral_effusion_prob_txt',
]

In [4]:
update_registered_converter(
    XGBClassifier,
    'XGBoostXGBClassifier',
    calculate_linear_classifier_output_shapes,
    convert_xgboost,
    options={"nocl": [True, False], "zipmap": [False, True]})

In [6]:
tensor_type_mapping = []
for col in ordinal_cols:
    tensor_type_mapping.append((col, Int64TensorType([1])))
for col in ratio_cols:
    tensor_type_mapping.append((col, FloatTensorType([1])))

In [7]:
model_onnx = convert_sklearn(
        pipeline_full,
        "fusion_model_1a",
        tensor_type_mapping,
        options={"zipmap": False}
        )
with open("../../onnx_models/fusion_model_1a.onnx", "wb") as f:
    f.write(model_onnx.SerializeToString())

/Users/steven/Documents/GitHub.nosync/mids-210-radiology-triage-models-spring24/.conda/lib/python3.11/site-packages/skl2onnx/operator_converters/imputer_op.py:110: RuntimeWarning: invalid value encountered in cast
  ar = np.array([op.missing_values]).astype(np.int64)


In [9]:
sess = rt.InferenceSession("../../onnx_models/fusion_model_1a.onnx", providers=['CPUExecutionProvider'])

In [10]:
sess